https://homepage.ntu.edu.tw/~karchung/miniconversations/MC.htm

In [1]:
# upload audio file
from google.colab import files
uploaded = files.upload()
path = next(iter(uploaded))

ModuleNotFoundError: No module named 'google.colab'

In [2]:
from ipywidgets import FileUpload
from IPython.display import display
upload = FileUpload()
display(upload)
print(upload)

FileUpload(value=(), description='Upload')

FileUpload(value=(), description='Upload')


Notes on usage:

- Make sure to [change runtime to GPU](https://www.tutorialspoint.com/google_colab/google_colab_using_free_gpu.htm).
- The transcript will be saved in Files, which you can find in the menu on the left.
- Change the number of speakers below if different from two.
- Pick a bigger model if you want more accuracy and a smaller model if you want the program to run faster ([more info](https://github.com/openai/whisper#available-models-and-languages)).
- If you know the language being spoken is English, then change language to 'English' as this improves performance.


High level overview of what's happening here:


1.   I'm using Open AI's Whisper model to seperate audio into segments and generate transcripts.
2.   I'm then generating speaker embeddings for each segments.
3.   Then I'm using agglomerative clustering on the embeddings to identify the speaker for each segment.   

Let me know if I can make it better!


In [1]:
path= ".\data\interview.mp3"

In [2]:
num_speakers = 2 #@param {type:"integer"}

language = 'English' #@param ['any', 'English']

model_size = 'large' #@param ['tiny', 'base', 'small', 'medium', 'large']


model_name = model_size
if language == 'English' and model_size != 'large':
  model_name += '.en'


In [3]:
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121

Looking in indexes: https://download.pytorch.org/whl/cu121


In [4]:
!pip install speechbrain==0.5.16

In [ ]:
 !pip install -q git clone https://github.com/speechbrain/speechbrain.git > /dev/null

  ERROR: Cannot unpack file /tmp/pip-unpack-0nd2ba5k/speechbrain.git (downloaded from /tmp/pip-req-build-myz5ss_x, content-type: text/html; charset=utf-8); cannot detect archive format
ERROR: Cannot determine archive format of /tmp/pip-req-build-myz5ss_x


In [5]:
!pip install -q git+https://github.com/openai/whisper.git
!pip install -q git+https://github.com/pyannote/pyannote-audio

In [6]:
# !pip install -q git+https://github.com/openai/whisper.git > /dev/null
# !pip install -q git+https://github.com/pyannote/pyannote-audio > /dev/null

import whisper
import datetime

import subprocess

import torch
import pyannote.audio
from pyannote.audio.pipelines.speaker_verification import PretrainedSpeakerEmbedding
embedding_model = PretrainedSpeakerEmbedding(
    "speechbrain/spkrec-ecapa-voxceleb",
    device=torch.device("cuda"))

# embedding_model = PretrainedSpeakerEmbedding(
#     "speechbrain/spkrec-ecapa-voxceleb",
#     device=torch.device("cpu"))

from pyannote.audio import Audio
from pyannote.core import Segment

import wave
import contextlib

from sklearn.cluster import AgglomerativeClustering
import numpy as np

The torchaudio backend is switched to 'soundfile'. Note that 'sox_io' is not supported on Windows.
The torchaudio backend is switched to 'soundfile'. Note that 'sox_io' is not supported on Windows.


In [7]:
import subprocess
if path[-3:] != 'wav':
  subprocess.call(['ffmpeg', '-i', path, 'audio.wav', '-y'])
  path = 'audio.wav'

In [8]:
model = whisper.load_model(model_size)

In [9]:
result = model.transcribe(path)
segments = result["segments"]

In [10]:
segments.pop(0)
segments

[{'id': 1,
  'seek': 0,
  'start': 4.44,
  'end': 10.9,
  'text': " So, today's expert teacher is Gabriella, a university English teacher from Leeds.",
  'tokens': [50587,
   407,
   11,
   965,
   311,
   5844,
   5027,
   307,
   50053,
   3505,
   11,
   257,
   5454,
   3669,
   5027,
   490,
   1456,
   5147,
   13,
   50910],
  'temperature': 0.0,
  'avg_logprob': -0.14630475251570993,
  'compression_ratio': 1.5614035087719298,
  'no_speech_prob': 0.02668028138577938},
 {'id': 2,
  'seek': 0,
  'start': 11.620000000000001,
  'end': 14.22,
  'text': ' Gabriella, hi, and thanks for joining us today.',
  'tokens': [50946,
   50053,
   3505,
   11,
   4879,
   11,
   293,
   3231,
   337,
   5549,
   505,
   965,
   13,
   51076],
  'temperature': 0.0,
  'avg_logprob': -0.14630475251570993,
  'compression_ratio': 1.5614035087719298,
  'no_speech_prob': 0.02668028138577938},
 {'id': 3,
  'seek': 0,
  'start': 14.58,
  'end': 15.6,
  'text': ' Thanks for having me.',
  'tokens': [51094

In [19]:
with contextlib.closing(wave.open(path,'r')) as f:
  frames = f.getnframes()
  rate = f.getframerate()
  duration = frames / float(rate)

print("frames", frames)
print("rate",rate)
print("duration", duration)

frames 8425539
rate 44100
duration 191.05530612244897


In [20]:
# audio = Audio( mono=True)
audio = Audio()

def segment_embedding(segment):
  start = segment["start"]
  # Whisper overshoots the end timestamp in the last segment
  end = min(duration, segment["end"])
  clip = Segment(start, end)
  waveform, sample_rate = audio.crop(path, clip)
   # Ensure waveform is mono
  if waveform.shape[0] > 1:
    waveform = waveform.mean(dim=0, keepdim=True)
  return embedding_model(waveform[None])

In [21]:
embeddings = np.zeros(shape=(len(segments), 192))

for i, segment in enumerate(segments):
  embeddings[i] = segment_embedding(segment)

embeddings = np.nan_to_num(embeddings)

In [22]:
clustering = AgglomerativeClustering(num_speakers).fit(embeddings)
labels = clustering.labels_
for i in range(len(segments)):
  segments[i]["speaker"] = 'SPEAKER ' + str(labels[i] + 1)

In [23]:
def time(secs):
  return datetime.timedelta(seconds=round(secs))

f = open("transcript.txt", "w")

for (i, segment) in enumerate(segments):
  if i == 0 or segments[i - 1]["speaker"] != segment["speaker"]:
    f.write("\n" + segment["speaker"] + ' ' + str(time(segment["start"])) + '\n')
  f.write(segment["text"][1:] + ' ')
f.close()

In [24]:
print(open('transcript.txt').read())


SPEAKER 1 0:00:04
So, today's expert teacher is Gabriella, a university English teacher from Leeds. Gabriella, hi, and thanks for joining us today. 
SPEAKER 2 0:00:15
Thanks for having me. 
SPEAKER 1 0:00:16
So, I have to confess, today's topic is something I am really bad at, listening. Most people say speaking is the most stressful part of learning a new language, but for me, with my B1 German, speaking isn't so bad. At least I'm in control of it, but listening, whoa, people speak so fast, and it's like my brain just shuts down. Am I just really strange and bad at listening? Tell me honestly, I can take it. 
SPEAKER 2 0:00:46
No, you're not strange. In fact, it's really common. You know, in exams, most people do pretty well in speaking compared with listening. Of course, exams are a different situation from real life, because in an exam, you can't ask for something to be repeated or explained. You usually have just one or maybe two opportunities to listen to the dialogue, and then i